# Manifold Learning and Graph Kernels
### Requirements 
Read [this article](http://www.dsi.unive.it/~atorsell/AI/graph/Unfolding.pdf) presenting a way to improve the disciminative power of graph kernels.
Choose one [graph kernel](http://www.dsi.unive.it/~atorsell/AI/graph/kernels.pdf) among

* Shortest-path Kernel
* Graphlet Kernel
* Random Walk Kernel
* Weisfeiler-Lehman Kernel


Choose one manifold learning technique among

* Isomap
* Diffusion Maps
* Laplacian Eigenmaps
* Local Linear Embedding


Compare the performance of an SVM trained on the given kernel, with or without the manifold learning step, on the following datasets:

* [PPI](http://www.dsi.unive.it/~atorsell/AI/graph/PPI.mat): this is a Protein-Protein Interaction dataset. Here proteins (nodes) are connected by an edge in the graph if they have a physical or functional association.
* [Shock](http://www.dsi.unive.it/~atorsell/AI/graph/Shock.mat): representing 2D shapes. Each graph is a skeletal-based representation of the differential structure of the boundary of a 2D shape.

**Note**: the datasets are contained in Matlab files. The variable `G` contains a vector of cells, one per graph. 
The entry `am` of each cell is the adjacency matrix of the graph.
The variable `labels`, contains the class-labels of each graph.	


### Imports

In [ ]:
import scipy.io as sio
import numpy as np
import threading
import pprint
import matplotlib.pyplot as plt
from copy import deepcopy
from collections import Counter
from sklearn import svm, manifold, preprocessing
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.svm import SVC

### Read the datasets
* Load the datasets
* Take the `G` and `labels` fields
* `G` consists in a list containing a list of arrays. Remove the external list. Afterwards take only the `am` field
* `labels` consists in many one-element lists. Create a single list removing one depth level


In [ ]:
BASE_URI = '/home/lorenzo/Dropbox/manifold-learning-and-graph-kernels/dataset/'
SHOCK_URI = 'SHOCK.mat'
PPI_URI = 'PPI.mat'
# `G` and `labels`
SHOCK = sio.loadmat(BASE_URI + SHOCK_URI)
SHOCK_G = SHOCK['G'][0] # read and get rid of external list
# print(SHOCK_G.dtype.names) 
# [('am', 'O'), ('nl', 'O'), ('al', 'O')]
SHOCK_G_adj = SHOCK_G['am'] # take only the adjacency matrix
SHOCK_labels = SHOCK['labels'].ravel() # read and get rid of useless 1-element lists
assert len(SHOCK_G_adj) == len(SHOCK_labels)
del SHOCK
PPI = sio.loadmat(BASE_URI + PPI_URI)
PPI_G = PPI['G'][0]
PPI_G_adj = PPI_G['am']
PPI_labels = PPI['labels'].ravel()
assert len(PPI_G_adj) == len(PPI_labels)
del PPI

# Weisfeiler-Lehman
### Algorithm

1. Multiset label determination
    * assign a multiset label $M_i(v)$ to each node $v \in G$ which consists of the multiset $\{l_{i-1}(u)$ | u is a neighbor of v$\}$
        * done in `determine_labels`
        * as per the paper, since our graphs are unlabelled, we use the node-degrees as starting labels for the node


2. Sorting each multiset
    * Sort elements in $M_i(v)$ in ascending order and concatenate them into a string $s_i(v)$
        * sorted and merged in `get_labels` 
    * Add $l_{i−1}(v)$ as a prefix to $s_i(v)$
        * done in `extend_labels`. Returns the string formatted as requested


3. Label compression
    * Map each string $s_i(v)$ to a compressed label using a hash function $f : \Sigma^∗ \rightarrow \Sigma$ such that $f(s_i(v)) = f(s_i (w))$ if and only if $s_i(v) = s_i(w)$
        * done in `compress_label` and `relabel`
    * As the first "hash", I use the highest degree of a node in all graphs, plus one (hence I'm sure that one is a hash instead of an original label


4. Relabeling
    * Set $l_i(v) = f(s_i(v))$ for all nodes in $G$ 
        * done in `relabel`


After having done all of the above, the similarity matrix for the N graphs is computed.
The `run()` method returns the similarity matrix containing the normalized values for all the graphs.

In [ ]:
class WeisfeilerLehman:
    '''
    Get a graph's starting labels (node degrees).
    graph --> actual graph
    '''
    def get_graph_starting_labels(self, graph):
        return np.dot(graph, np.ones((len(graph), 1)))
    
    
    '''
    Get the starting labels for all the graphs (node degrees)
    {index of the graph in the graphs list : array representing starting label}
    '''
    def get_all_starting_labels(self):
        starting_labels = {g : self.get_graph_starting_labels(self.graphs[g]) 
                                                              for g in range(self.n)}
        return starting_labels
    
    
    '''
    Get the highest degree of a node throughout all graphs
    '''
    def get_max_global_degree(self):
        return max([max(v) for _, v in self.get_all_starting_labels().items()])
    
    
    '''
    Get the neighbors of a node
    g --> index of the graph
    node --> index of the node
    neighbors --> list with indices of the neighbors
    '''
    def get_neighbors(self, g, node):
        graph = self.graphs[g]
        neighbors = [j for j in range(len(graph)) if graph[node][j] == 1]
        return neighbors
    
    
    '''
    Get updated labels for a node as per (1)
    g --> index of the graph
    node --> index of the node in the graph
    
    Take the labels of the neighbors of a node, sort them, merge them into an unique string
    '''
    def get_labels(self, g, node):
        new_label = sorted([self.labels[g][i] for i in self.get_neighbors(g, node)])
        new_label = ''.join(str(int(i)) for i in new_label)
        return new_label
    
    
    '''
    Compute the new multiset of labels of each node in a graph.
    Return a dictionary in which the key is the index of the node and the value 
    is the string returned from the `get_labels` function
    g --> index of the graph in the graphs array
    '''
    def determine_labels(self, g):
        new_labels = {k : self.get_labels(g, k) for k in range(len(self.graphs[g]))}
        return new_labels
    
    
    '''
    Return the string obtained from the sorted multiset 
    g --> index of the graph in the array
    '''
    def extend_labels(self, g, new_labels):
        for k in new_labels: # new_labels is a dict
            new_labels[k] = self.labels[g][k] + new_labels[k]
        return new_labels    
    
    
    '''
    Compress a label if it has not been compressed already
    {long_label : compressed_index}
    '''
    def compress_label(self, label):
        if label not in self.compressed_labels:
            self.compressed_labels[label] = str(self.compressed_index)
            self.compressed_index += 1
        return self.compressed_labels[label]
    
    
    '''
    Relabel all the nodes in a graph
    '''
    def relabel(self, g, new_labels):
        assert len(new_labels) == len(self.labels[g])
        for i in range(len(new_labels)):
            self.labels[g][i] = self.compress_label(new_labels[i])
        
    
    '''
    Count the original node labels: return a list with the number of occurrences per each label
    g --> index of the graph
    [0, 1, 2, 3, 1] --> 0 nodes with label 0, 1 node with label 1, ..., 1 node with label 4
    '''
    def count_original_node_labels(self, g):
        phi = []
        ol = list(map(int, self.original_labels[g]))
        c = Counter(ol)
        phi = np.zeros(max(ol)+1)
        for k in range(max(ol)+1):
            if k in c:
                phi[k] = c[k]
        return phi
    
    
    '''
    Count node labels at current iteration: return a list with the number of occurrences per each label
    g --> index of the graph
    '''
    def count_node_labels_at_current_iteration(self, g):
        l = list(map(int, [i for _, i in self.compressed_labels.items()]))
        c = Counter(self.labels[g])
        m = max(int(i) for i in c)
        phi = np.zeros(m+1)
        for k in range(m+1):
            if str(k) in c:
                phi[k] = c[str(k)]
        return phi
    
    
    '''
    Prepare feature vectors for the dot product. Make the shorter ones as long as the long ones,
    concatenate, and so on.
    g1, g2 --> indices of two graphs
    '''
    def prepare_feature_vectors(self, g1, g2):
        l1 = self.count_labels[g1]
        l2 = self.count_labels[g2]
        tot1 = []
        tot2 = []
        for i in range(self.h+1):
            l = min(len(l1[i]), len(l2[i]))
            a1 = l1[i][:l]
            a2 = l2[i][:l]
            tot1 = np.concatenate((tot1, a1), axis=0)
            tot2 = np.concatenate((tot2, a2), axis=0)
        return tot1, tot2
    
    
    '''
    Normalize similarity matrix: sum per rows must be = 1
    '''
    def normalize_sim_matrix(self):
        self.pairwise_sim_matrix = preprocessing.normalize(self.pairwise_sim_matrix)
        return self.pairwise_sim_matrix
    
    
    '''
    Pairwise similarities between all the graphs (unnormalized)
    '''
    def pairwise_similarities(self):
        for i in range(self.n):
            for j in range(i, self.n):
                g1, g2 = self.prepare_feature_vectors(i, j)
                dotp = np.dot(g1, g2)
                self.pairwise_sim_matrix[i][j] = dotp
                self.pairwise_sim_matrix[j][i] = dotp
        self.normalize_sim_matrix()
        return self.pairwise_sim_matrix
    
    
    '''
    Run the whole algorithm: steps 1, 2, 3, 4
    '''
    def run(self):
        for i in range(self.h):
            for g in range(self.n): # g is the index of the graph in the array of graphs
                new_labels = self.determine_labels(g)
                new_labels = self.extend_labels(g, new_labels)
                self.relabel(g, new_labels)
                self.count_labels[g][i+1] = self.count_node_labels_at_current_iteration(g)
        return self.pairwise_similarities()

    
    '''
    Initialize everything
    '''
    def __init__(self, graphs, h):
        self.n = len(graphs)
        self.graphs = graphs
        self.h = h
        self.labels = self.get_all_starting_labels()
        self.labels = { 
                        index : [str(int(degree)) for degree in self.labels[index].ravel()] 
                                                  for index in self.labels 
                      }
        self.original_labels = deepcopy(self.labels)
        self.compressed_index = int(self.get_max_global_degree()[0]) + 1
        self.compressed_labels = {}
        self.count_labels = {} # {iter1 : {graph1 : count1, graph2 : count2, ...}, ...}
        for i in range(self.n):
            self.count_labels[i] = {}
            self.count_labels[i][0] = self.count_original_node_labels(i)            
        del self.original_labels
        self.pairwise_sim_matrix = np.zeros((self.n, self.n))
        
    


### Compute similarity matrices for PPI and SHOCK

In [ ]:
wl_PPI = WeisfeilerLehman(PPI_G_adj,4)
wl_SHOCK = WeisfeilerLehman(SHOCK_G_adj, 4)

t1 = threading.Thread(name="PPI", target=wl_PPI.run)
t2 = threading.Thread(name="SHOCK", target=wl_SHOCK.run)

threads = [t1, t2]
for t in threads:
    t.start()
for t in threads:
    t.join()

similarity_matrices = [wl_SHOCK.pairwise_sim_matrix, wl_PPI.pairwise_sim_matrix]
labels = [SHOCK_labels, PPI_labels]
dn = ['SHOCK', 'PPI']

### Plot the distance matrices

In [ ]:
dms = []
for i in range(len(similarity_matrices)):
    pd = pairwise_distances(similarity_matrices[i], metric='euclidean')
    dms.append(pd)
    plt.imshow(pd, cmap='Blues')
    plt.colorbar()
    plt.show()

### SVMs without manifold learning step

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True) 
clf = svm.SVC(kernel="linear")
for i in range(len(dms)):
    cv_score = cross_val_score(clf, dms[i], labels[i], cv = kfold)
    print(f'DATASET: {dn[i]}\nMinimum: {np.min(cv_score)}\nMean: {np.mean(cv_score)}\nMax: {np.max(cv_score)}')
    print(f'Standard deviation: {np.std(cv_score)}\n')

### SVMs with Isomap manifold learning

In [ ]:
pp = pprint.PrettyPrinter(indent=2)
results_isomap = {}
best = [0, 0]
worst = [1, 1] 

results_isomap[dn[0]] = {}
results_isomap[dn[1]] = {}


for n_neigh in range(2, 25):
    for n_comp in range(2, 10):
        for i in range(len(dn)):
            p = manifold.Isomap(n_neigh, n_comp).fit_transform(dms[i])
            cv_score = cross_val_score(clf, p, labels[i], cv = kfold)
            if np.mean(cv_score) > best[i]:
                results_isomap[dn[i]]['best'] = [{
                    'min' : np.min(cv_score),
                    'mean' : np.mean(cv_score),
                    'max' : np.max(cv_score),
                    'stddev' : np.std(cv_score)
                }, (n_neigh, n_comp)]
                best[i] = np.mean(cv_score)
            if np.mean(cv_score) < worst[i]:
                results_isomap[dn[i]]['worst'] = [{
                    'min' : np.min(cv_score),
                    'mean' : np.mean(cv_score),
                    'max' : np.max(cv_score),
                    'stddev' : np.std(cv_score)
                }, (n_neigh, n_comp)]
                worst[i] = np.mean(cv_score)

In [ ]:
pp.pprint(results_isomap)

### SVMs with LLE manifold learning

In [ ]:
results_lle = {}
best = [0, 0]
worst = [1, 1] 

results_lle[dn[0]] = {}
results_lle[dn[1]] = {}


for n_neigh in range(2, 25):
    for n_comp in range(2, 10):
        for i in range(len(dn)):
            p = manifold.LocallyLinearEmbedding(n_neigh, n_comp).fit_transform(dms[i])
            cv_score = cross_val_score(clf, p, labels[i], cv = kfold)
            if np.mean(cv_score) > best[i]:
                results_lle[dn[i]]['best'] = [{
                    'min' : np.min(cv_score),
                    'mean' : np.mean(cv_score),
                    'max' : np.max(cv_score),
                    'stddev' : np.std(cv_score)
                }, (n_neigh, n_comp)]
                best[i] = np.mean(cv_score)
            if np.mean(cv_score) < worst[i]:
                results_lle[dn[i]]['worst'] = [{
                    'min' : np.min(cv_score),
                    'mean' : np.mean(cv_score),
                    'max' : np.max(cv_score),
                    'stddev' : np.std(cv_score)
                }, (n_neigh, n_comp)]
                worst[i] = np.mean(cv_score)

In [ ]:
pp.pprint(results_lle)